In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd
import numpy as np
from scipy.special import softmax

c:\Users\esteb\miniconda3\envs\deep_learning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

In [3]:
def map_to_sentiment(score_idx):
    if score_idx <= 1:
        return 'negative'
    elif score_idx == 2:
        return 'neutral'
    else:
        return 'positive'

In [4]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    with torch.no_grad():
        logits = model(**inputs).logits
    probs = softmax(logits.numpy()[0])
    pred_idx = np.argmax(probs)
    return map_to_sentiment(pred_idx)

In [5]:
comments = pd.read_csv("../data/processed/comments_processed.csv")
comments = comments.dropna(subset=['body'])

In [6]:
comments["sentiment"] = comments["body"].apply(lambda x: predict_sentiment(str(x)))

In [7]:
comments.to_csv("../data/analice/comments_labeled.csv", index=False)

In [8]:
df = pd.read_csv('../data/analice/comments_labeled.csv')
df['sentiment'].value_counts()

sentiment
negative    354
positive    307
neutral     139
Name: count, dtype: int64